In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
import scipy as sp
import datetime
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import dask
import math
import warnings

In [ ]:
# Suppress warning, there was a continuous numpy warning because of NaN values
warnings.filterwarnings('ignore')

# Set the facecolor of the plots so that the background is not transparent when saved
plt.rcParams['axes.facecolor']='white'
plt.rcParams['savefig.facecolor']='white'

# Open all files saved using the data_import file into one large dataset
#all_data = xr.open_mfdataset('*ERB.nc',chunks = {'lat':10, 'lon':10, 'time':10})

# Initialize a few variables
inp = ' '
lat_slice = []
lon_slice = []
time_slice = []
parsed = []
times1 = 0
times2 = 0
lats = []
lons = []
times = []
seasons = []
titles = []
all_data = []
angles = []
angle_count = 0
items = 1

# Prompt which path to use to plot
while inp.upper() not in ('TEMPORAL','SPATIAL','QUIT'):
    inp = str(input('Would you like to make a spatial plot or temporal plot?'))

################ Spatial Plot ################

    if inp.upper() == 'SPATIAL':
        # Open all files saved using the data_import file into one large dataset
        all_data = xr.open_mfdataset('*0.0_ERB.nc',chunks = {'lat':10, 'lon':10, 'time':10})
        
        ########## Latitudes ##########
        
        # Continue until intentionally broken: All inputs in this function make use of this so that invalid inputs cannot be passed forward
        while True:
        
            try:
                inp = input('What latitudes would you like to look at? Please enter a comma separated list with pairs next to each other.\nEnter all for no slice.')
                
                # If all latitudes are chosed via the all input, add all the data to a list 
                if inp.lower() == 'all':
                    lat_slice.append(all_data)
                    # Append the upper and lower limits of the spatial plot to a list for titleing
                    lats.append('-81')
                    lats.append('81')
                    break
                    
                # allow the ability to quit (this is prevalent throughout the code and will only be commented here)
                elif inp.lower() == 'quit':
                    break
                    
                # If specific latitudes are chosen make a list of those latitudes
                else:
                    lats = inp.split(',')
                    # Check if any latitudes are outside the available range and change them
                    for i in range(len(lats)):
                        lats[i] = int(lats[i])
                        if lats[i] > 81:
                            lats[i] = 81
                        elif lats[i] < -81:
                            lats[i] = -81

                    # Check if pairs have been provided and slice the overall datset accordingly then add the slices to a list
                    if len(lats)%2 == 0:
                        for i in range(int(len(lats)/2)):
                            j = 2*i
                            lat_slice.append(all_data.sel(lat = slice(lats[j],lats[j+1])))
                        break

                    # Reprompt the question because a spatial plot needs an upper and lower spatial limit AKA pairs
                    else:
                        print('Pairs are required. Please try again.')

            except ValueError:
                print('Unrecognized response. Please try again.')
            
        # Delete the all_data variable to save memory
        del all_data
        
        if inp.lower() == 'quit':
            break
                
        ########## Longitudes ##########
        
        while True:
        
            try:
                inp = input('What longitudes would you like to look at? Please enter a comma separated list with pairs next to each other.\nEnter all for no slice.')
                
                # If all longitudes are chosen, no slicing is necessary and the latitude slices can be passed down
                if inp.lower() == 'all':
                    lon_slice = lat_slice
                    # Append bounds to list for titleing
                    lons.append('-180')
                    lons.append('180')
                    break
                    
                elif inp.lower() == 'quit':
                    break
                    
                # If longitude bounds are set, create a list with the bounds
                else:
                    lons = inp.split(',')
                    for i in range(len(lons)):
                        lons[i] = int(lons[i])

                    # Check to ensure pairs and then slice each latitude set for every pair of longitudes
                    if len(lons)%2 == 0:
                        for lat in range(len(lat_slice)):
                            for lon in range(int(len(lons)/2)):
                                j = 2*lon
                                lon_slice.append(lat_slice[lat].sel(lon = slice(lons[j],lons[j+1])))
                        break

                    else:
                        print('Pairs are required. Please try again')

            except ValueError:
                print('Unrecognized response. Please try again.')

        del lat_slice
        
        if inp.lower() == 'quit':
            break
        
        ########## Time ##########
                
        while True:
            inp = input('What time would you like to look at? Enter format as YYYY,MM,DD-YYYY,MM,DD. Enter all for no slice.')
            
            try:
                if inp.lower() == 'quit':
                    break

                # Slice all lon-lat slices for desired time
                elif inp.lower() != 'all':
                    # Get list with the various numbers provided in the input
                    times = inp.replace('-',',')
                    times = times.split(',')
                    for i in range(len(times)):
                        times[i] = int(times[i])

                    # Turn input values into datetimes
                    times1 = datetime.datetime(times[0],times[1],times[2])
                    times2 = datetime.datetime(times[3],times[4],times[5]) + datetime.timedelta(days = 1)
                    
                    # Add datetimes to a list for titleing
                    times = []
                    times.append(times1.strftime('%Y-%m-%d'))
                    times.append(times2.strftime('%Y-%m-%d'))


                    # Slice all lat-lon slices
                    for i in range(len(lon_slice)):
                        time_slice.append(lon_slice[i].sel(time = slice(str(times1),str(times2))))
                        

                    del lon_slice

                    break

                # If all time is chosen, no slicing is necessary, but the first and last date are added to a list to title
                elif inp.lower() == 'all':
                    for i in range(len(lon_slice)):
                        time_slice.append(lon_slice[i])
                        times.append(np.datetime_as_string(time_slice[0].time.isel(time = 0).values, unit = 'D'))
                        times.append(np.datetime_as_string(time_slice[0].time.isel(time = -1).values, unit = 'D'))

                    del lon_slice

                    break

                else:
                    print('Unrecognized response.')
                    
            except ValueError:
                print('Unrecognized response.')
                
            
        if inp.lower() == 'quit':
            break
            
            
        while True:
                
            inp = input('Would you like to view seasons?')
            
            if inp.lower() in ('y','yes'):
                for i in range(len(parsed)):
                    # Group every lat-lon-time slice into seasons if seasons are desired
                    time_slice[i].groupby('time.season')
                    
                    
                while True:
                    inp = input('Which seasons would you like to view? Enter a comma separated list.')
                    
                    if inp.lower() == 'quit':
                        break
                        
                    # Make sure only seasons were submitted and make them lower case
                    elif (inp.split(',')[0].lower() in ('spring', 'summer', 'fall', 'winter')):
                        season_choice = inp.split(',')
                        for i in range(len(season_choice)):
                            season_choice[i] = season_choice[i].lower()
                            
                        # Check which seasons were selected, make slices accordingly, take the average with respect to time, and add the season to a list for titles
                            
                        if all(item in season_choice for item in ('spring', 'summer', 'fall', 'winter')):
                            for i in range(len(time_slice)):
                                parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'DJF').mean('time'))
                                parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'MAM').mean('time'))
                                parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'JJA').mean('time'))
                                parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'SON').mean('time'))
                            seasons.append('Dec-Feb')
                            seasons.append('Mar-May')
                            seasons.append('Jun-Aug')
                            seasons.append('Sep-Nov')
                            
                        elif all(item in season_choice for item in ('spring', 'summer', 'fall')):
                            for i in range(len(time_slice)):
                                parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'MAM').mean('time'))
                                parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'JJA').mean('time'))
                                parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'SON').mean('time'))
                            seasons.append('Mar-May')
                            seasons.append('Jun-Aug')
                            seasons.append('Sep-Nov')

                        elif all(item in season_choice for item in('spring', 'summer', 'winter')):
                            for i in range(len(time_slice)):
                                parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'DJF').mean('time'))
                                parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'MAM').mean('time'))
                                parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'JJA').mean('time'))
                            seasons.append('Dec-Feb')
                            seasons.append('Mar-May')
                            seasons.append('Jun-Aug')
  
                        elif all(item in season_choice for item in('spring', 'fall', 'winter')):
                            for i in range(len(time_slice)):
                                parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'DJF').mean('time'))
                                parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'MAM').mean('time'))
                                parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'SON').mean('time'))
                            seasons.append('Dec-Feb')
                            seasons.append('Mar-May')
                            seasons.append('Sep-Nov')
                           
                        elif all(item in season_choice for item in ('summer', 'fall', 'winter')):
                            for i in range(len(time_slice)):
                                parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'DJF').mean('time'))
                                parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'JJA').mean('time'))
                                parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'SON').mean('time'))
                            seasons.append('Dec-Feb')
                            seasons.append('Jun-Aug')
                            seasons.append('Sep-Nov')
                           
                        elif all(item in season_choice for item in ('fall', 'winter')):
                            for i in range(len(time_slice)):
                                parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'DJF').mean('time'))
                                parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'SON').mean('time'))
                            seasons.append('Dec-Feb')
                            seasons.append('Sep-Nov')
                          
                        elif all(item in season_choice for item in ('summer', 'winter')):
                            for i in range(len(time_slice)):
                                parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'DJF').mean('time'))
                                parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'JJA').mean('time'))
                            seasons.append('Dec-Feb')
                            seasons.append('Jun-Aug')
                           
                        elif all(item in season_choice for item in ('summer', 'fall')):
                            for i in range(len(time_slice)):
                                parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'JJA').mean('time'))
                                parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'SON').mean('time'))
                            seasons.append('Jun-Aug')
                            seasons.append('Sep-Nov')
                       
                        elif all(item in season_choice for item in ('spring', 'winter')):
                            for i in range(len(time_slice)):
                                parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'DJF').mean('time'))
                                parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'MAM').mean('time'))
                            seasons.append('Dec-Feb')
                            seasons.append('Mar-May')
                        
                        elif all(item in season_choice for item in ('spring','fall')):
                            for i in range(len(time_slice)):
                                parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'MAM').mean('time'))
                                parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'SON').mean('time'))
                            seasons.append('Mar-May')
                            seasons.append('Sep-Nov')
                            
                        elif all(item in season_choice for item in ('spring', 'summer')):
                            for i in range(len(time_slice)):
                                parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'MAM').mean('time'))
                                parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'JJA').mean('time'))
                            seasons.append('Mar-May')
                            seasons.append('Jun-Aug')
                            
                        elif ('spring') in season_choice:
                            for i in range(len(time_slice)):
                                parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'MAM').mean('time'))
                            seasons.append('Mar-May')
                            
                        elif ('summer') in season_choice:
                            for i in range(len(time_slice)):
                                parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'JJA').mean('time'))
                            seasons.append('Jun-Aug')
                            
                        elif ('fall') in season_choice:
                            for i in range(len(time_slice)):
                                parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'SON').mean('time'))
                            seasons.append('Sep-Nov')
                            
                        elif ('winter') in season_choice:
                            for i in range(len(time_slice)):
                                parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'DJF').mean('time'))
                            seasons.append('Dec-Feb')
                            
                        del time_slice
                        
                        break
                        
                    if inp.lower() == 'quit':
                        break
                    
            # Take the time-average for all slices if no seasons are desired
            elif inp.lower() in ('n','no'):
                for i in range(len(time_slice)):
                    parsed.append(time_slice[i].mean('time'))
                    
                del time_slice
            
                
            else:
                print('Unrecognized response.')
                
            if inp.lower() == 'quit':
                break
                
            ########## Plot ##########
            
            while True:
                inp = input('What would you like to plot?\n-Longwave\n-Shortwave Cumulative\n-Shortwave Day\n-Shortwave and Longwave\n-Shortwave Day and Longwave\n-Total\n')
                
                # Turn all saved variables for titles into strings
                for i in range(len(lats)):
                    lats[i] = str(lats[i])
                for i in range(len(lons)):
                    lons[i] = str(lons[i])

                # Use this title method if seasons are present
                if len(seasons)>0:
                    # Iterate through every other index of latitudes and longitudes and all indices of seasons
                    for i in range(int(len(lats)/2)):
                        for k in range(int(len(lons)/2)):
                            for j in range(len(seasons)):
                                # Use intermittent variables so that the loop truly does every other lat and lon
                                l = i*2
                                m = k*2
                                # Create a list of titles
                                #titles.append('Latitudes: '+lats[l]+' - '+lats[l+1]+', Longitudes: '+lons[m]+' - '+lons[m+1]+'\nSeason: '+seasons[j]+', Time Range: '+times[0]+'-'+times[1])
                                #titles.append('Latitudes: '+lats[l]+', '+lats[l+1]+'\nSeason: '+seasons[j])
                                titles.append(seasons[j])
                # Exact same as the previous but no seasons
                else:
                    for i in range(int(len(lats)/2)):
                        for k in range(int(len(lons)/2)):
                            l = i*2
                            m = k*2
                            #titles.append('Latitudes: '+lats[l]+' - '+lats[l+1]+', Longitudes: '+lons[m]+' - '+lons[m+1]+', Time Range: '+times[0]+'-'+times[1])
                            titles.append('Latitudes: '+lats[l]+', '+lats[l+1])

                
                if inp.lower() == 'quit':
                    break
                
                
                # Almost all plots are the same so comments will only be on those different from this
                elif inp.lower() == 'longwave':
                    
                    # Set up 2 columns if there are enough plots, otherwise just 1 or the amount of plots available
                    c = 2 if len(parsed)>2 else len(parsed)
                    # Create enough rows to compensate the amount of plots that need to be made
                    r = math.ceil(len(parsed)/c)
                    
                    # Set up the plots and figure
                    f, p = plt.subplots(nrows=r,ncols=c,figsize=(15,15 if len(parsed) > 4 else 10),subplot_kw=dict(projection=ccrs.PlateCarree()))
                    
                    # Figure for more than one plot
                    if len(parsed) > 1:
                        # Flatten the shape of the plots so that it can be iterated through easily
                        p = p.flatten()
                        
                        # Plot every data slice that has been saved
                        for i in range(len(parsed)):
                            
                            # Set up a color plot with coastlines, and a color bar that ranges from 0 to 300 with steps of 20 and an arrow at the top, the give it its title
                            im = p[i].contourf(parsed[i].lon,parsed[i].lat,parsed[i].LWrad.load().interpolate_na(dim='lon'),transform = ccrs.PlateCarree(),
                                levels=np.arange(0,320,20),
                                          extend='max')
                            p[i].coastlines()
                            p[i].set_title(titles[i],fontsize=20)

                        # Set the colorbar and add its label
                        cbar_ax = f.add_axes([1,.15,.05,.7])
                        cbar = f.colorbar(im,cax=cbar_ax)
                        cbar.set_label('Radiance (W '+'$m^{-2}$ '+'$sr^{-1}$)',fontsize=20)
                        cbar.ax.tick_params(labelsize=16)

                    
                        
                    else:
                        # Same as the iterative process but with only a single plot
                        im = p.contourf(parsed[0].lon,parsed[0].lat,parsed[0].LWrad.load().interpolate_na(dim='lon'),transform = ccrs.PlateCarree(),
                            levels=np.arange(0,320,20),
                                          extend='max')
                        p.coastlines()
                        p.set_title(titles[0],fontsize=20)

                        cbar_ax = f.add_axes([1,.15,.05,.7])
                        cbar = f.colorbar(im,cax=cbar_ax)
                        cbar.set_label('Radiance (W '+'$m^{-2}$ '+'$sr^{-1}$)',fontsize=20)
                        cbar.ax.tick_params(labelsize=16)

                    # Put the plots as close as possible within the figure
                    plt.tight_layout()
                        
                    # Give the whole figure a name
                    # Set the location of the figure title based on the amount of plots
                    plt.suptitle('Longwave Radiance',fontsize = 30,y = .88 if len(parsed) < 3 else 1.05)
                    
                    
                elif inp.lower() == 'shortwave cumulative':
                    
                    c = 2 if len(parsed)>2 else len(parsed)
                    r = math.ceil(len(parsed)/c)
                    
                    f, p = plt.subplots(nrows=r,ncols=c,figsize=(15,15 if len(parsed) > 4 else 10),subplot_kw=dict(projection=ccrs.PlateCarree()))
                    
                    if len(parsed) > 1:
                        p = p.flatten()

                        for i in range(len(parsed)):

                            im = p[i].contourf(parsed[i].lon,parsed[i].lat,parsed[i].SWrad.load().interpolate_na(dim='lon'),transform = ccrs.PlateCarree(),
                                levels=np.arange(0,320,20),
                                          extend='max')
                            p[i].coastlines()
                            p[i].set_title(titles[i],fontsize=20)

                        cbar_ax = f.add_axes([1,.15,.05,.7])
                        cbar = f.colorbar(im,cax=cbar_ax)
                        cbar.set_label('Radiance (W '+'$m^{-2}$ '+'$sr^{-1}$)',fontsize=20)
                        cbar.ax.tick_params(labelsize=16)

                      
                        
                    else:
                        im = p.contourf(parsed[0].lon,parsed[0].lat,parsed[0].SWrad.load().interpolate_na(dim='lon'),transform = ccrs.PlateCarree(),
                            levels=np.arange(0,320,20),
                                          extend='max')
                        p.coastlines()
                        p.set_title(titles[0],fontsize=20)

                        cbar_ax = f.add_axes([1,.15,.05,.7])
                        cbar = f.colorbar(im,cax=cbar_ax)
                        cbar.set_label('Radiance (W '+'$m^{-2}$ '+'$sr^{-1}$)',fontsize=20)
                        cbar.ax.tick_params(labelsize=16)

                    plt.tight_layout()
                        
                    plt.suptitle('Shortwave Radiance',fontsize = 30,y = .88 if len(parsed) < 3 else 1.05)
                  
                        
                elif inp.lower() == 'shortwave day':
                    
                    c = 2 if len(parsed)>2 else len(parsed)
                    r = math.ceil(len(parsed)/c)
                    
                    f, p = plt.subplots(nrows=r,ncols=c,figsize=(15,15 if len(parsed) > 4 else 10),subplot_kw=dict(projection=ccrs.PlateCarree()))
                    
                    if len(parsed) > 1:
                        p = p.flatten()

                        for i in range(len(parsed)):

                            im = p[i].contourf(parsed[i].lon,parsed[i].lat,parsed[i].SWrad_day.load().interpolate_na(dim='lon'),transform = ccrs.PlateCarree(),
                                levels=np.arange(0,320,20),
                                          extend='max')
                            p[i].coastlines()
                            p[i].set_title(titles[i],fontsize=20)

                        cbar_ax = f.add_axes([1,.15,.05,.7])
                        cbar = f.colorbar(im,cax=cbar_ax)
                        cbar.set_label('Radiance (W '+'$m^{-2}$ '+'$sr^{-1}$)',fontsize=20)
                        cbar.ax.tick_params(labelsize=16)

                      
                        
                    else:
                        im = p.contourf(parsed[0].lon,parsed[0].lat,parsed[0].SWrad_day.load().interpolate_na(dim='lon'),transform = ccrs.PlateCarree(),
                            levels=np.arange(0,320,20),
                                          extend='max')
                        p.coastlines()
                        p.set_title(titles[0],fontsize=20)

                        cbar_ax = f.add_axes([1,.15,.05,.7])
                        cbar = f.colorbar(im,cax=cbar_ax)
                        cbar.set_label('Radiance (W '+'$m^{-2}$ '+'$sr^{-1}$)',fontsize=20)
                        cbar.ax.tick_params(labelsize=16)

                    plt.tight_layout()
                        
                    plt.suptitle('Shortwave Daytime Radiance',fontsize = 30,y = 88 if len(parsed) < 3 else 1.05)
                       

                # This is a different plot
                elif inp.lower() == 'total':
                    
                    c = 2 if len(parsed)>2 else len(parsed)
                    r = math.ceil(len(parsed)/c)
                    
                    f, p = plt.subplots(nrows=r,ncols=c,figsize=(15,15 if len(parsed) > 4 else 10),subplot_kw=dict(projection=ccrs.PlateCarree()))
                    
                    if len(parsed) > 1:
                        p = p.flatten()

                        for i in range(len(parsed)):
                            # The only difference is in the below line, two variables are added together to plot a third instead of simply plotting one variable
                            im = p[i].contourf(parsed[i].lon,parsed[i].lat,(parsed[i].LWrad.load().interpolate_na(dim='lon')+parsed[i].SWrad_day.load().interpolate_na(dim='lat')),transform = ccrs.PlateCarree(),
                                levels=np.arange(0,320,20),
                                          extend='max')
                            p[i].coastlines()
                            p[i].set_title(titles[i],fontsize=20)

                        cbar_ax = f.add_axes([1,.15,.05,.7])
                        cbar = f.colorbar(im,cax=cbar_ax)
                        cbar.set_label('Radiance (W '+'$m^{-2}$ '+'$sr^{-1}$)',fontsize=20)
                        cbar.ax.tick_params(labelsize=16)

                        
                        
                    else:
                        im = p.contourf(parsed[0].lon,parsed[0].lat,(parsed[0].LWrad.load().interpolate_na(dim='lon')+parsed[0].SWrad_day.load().interpolate_na(dim='lat')),transform = ccrs.PlateCarree(),
                            levels=np.arange(0,320,20),
                                          extend='max')
                        p.coastlines()
                        p.set_title(titles[0],fontsize=20)

                        cbar_ax = f.add_axes([1,.15,.05,.7])
                        cbar = f.colorbar(im,cax=cbar_ax)
                        cbar.set_label('Radiance (W '+'$m^{-2}$ '+'$sr^{-1}$)',fontsize=20)
                        cbar.ax.tick_params(labelsize=16)

                    plt.tight_layout()
                        
                    plt.suptitle('Total Radiance',fontsize = 30,y = .88 if len(parsed) < 3 else 1.05)
                     

                # This plot is the most different. For every slice, two plots are made
                elif inp.lower() == 'shortwave and longwave':
                    # A minimum of two plots are made (shortwave and longwave for a single slice) so 2 columns are alwats necessary
                    c = 2
                    # Compensate for total slices by calculating necessary rows
                    r = math.ceil(2*len(parsed)/c)
                    
                    f, p = plt.subplots(nrows=r,ncols=c,figsize=(15,15 if len(parsed) > 4 else 10),subplot_kw=dict(projection=ccrs.PlateCarree()))
                    
                    if len(parsed) > 1:
                        p = p.flatten()

                        # Iterate through 2 times the amount of slices present so that 2 plots may be made for each slice
                        for i in range(2*len(parsed)):

                            # When the index is even plot longwave
                            # The true index will be half of the passed index (0->0,2->1,4->2)
                            if i%2 == 0:
                                im = p[i].contourf(parsed[int(i/2)].lon,parsed[int(i/2)].lat,parsed[int(i/2)].LWrad.load().interpolate_na(dim='lon'),transform = ccrs.PlateCarree(),
                                    levels=np.arange(0,100,10),
                                          extend='max',
                                                   cmap = 'cividis')
                                p[i].coastlines()
                                p[i].set_title('Longwave\n'+titles[int(i/2)],fontsize=20)

                            # When the index is even plot shortwave
                            # The true index is the passed half the passed index rounded down [I did it more complicated than this] (1->0, 3->1, 5->2)
                            else:
                                im = p[i].contourf(parsed[i-int(math.ceil(i/2))].lon,parsed[i-int(math.ceil(i/2))].lat,parsed[i-int(math.ceil(i/2))].SWrad.load().interpolate_na(dim='lat'),transform = ccrs.PlateCarree(),
                                    levels=np.arange(0,100,10),
                                          extend='max',
                                                  cmap = 'cividis')
                                p[i].coastlines()
                                p[i].set_title('Shortwave\n'+titles[i-int(math.ceil(i/2))],fontsize=20)

                        cbar_ax = f.add_axes([1,.15,.05,.7])
                        cbar = f.colorbar(im,cax=cbar_ax)
                        cbar.set_label('Radiance (W '+'$m^{-2}$ '+'$sr^{-1}$)',fontsize=20)
                        cbar.ax.tick_params(labelsize=16)

                      
                    # Same as iterative plot but without math
                    else:
                        im = p[0].contourf(parsed[0].lon,parsed[0].lat,parsed[0].LWrad.load().interpolate_na(dim='lon'),transform = ccrs.PlateCarree(),
                            levels=np.arange(0,100,10),
                                          extend='max')
                        p[0].coastlines()
                        p[0].set_title('Longwave\n'+titles[0],fontsize=20)
                        
                        im = p[1].contourf(parsed[0].lon,parsed[0].lat,parsed[0].SWrad.load().interpolate_na(dim='lon'),transform = ccrs.PlateCarree(),
                            levels=np.arange(0,100,10),
                                          extend='max')
                        p[1].coastlines()
                        p[1].set_title('Shortwave\n'+titles[0],fontsize=20)

                        cbar_ax = f.add_axes([1,.15,.05,.7])
                        cbar = f.colorbar(im,cax=cbar_ax)
                        cbar.set_label('Radiance (W '+'$m^{-2}$ '+'$sr^{-1}$)',fontsize=20)
                        cbar.ax.tick_params(labelsize=16)

                    plt.tight_layout()
                        
                    plt.suptitle('Shortwave and Longwave Radiance Comparison',fontsize = 30,y = .88 if len(parsed) < 3 else 1.05)
                       
                # This plot is the most different. For every slice, two plots are made
                elif inp.lower() == 'shortwave day and longwave':
                    # A minimum of two plots are made (shortwave and longwave for a single slice) so 2 columns are alwats necessary
                    c = 2 if len(parsed) > 1 else 1
                    # Compensate for total slices by calculating necessary rows
                    r = math.ceil(2*len(parsed)/c)
                    
                    f, p = plt.subplots(nrows=r,ncols=c,figsize=(15 if len(parsed) > 4 else 10,15 if len(parsed) > 4 else 10),subplot_kw=dict(projection=ccrs.PlateCarree()))
                    
                    if len(parsed) > 1:
                        p = p.flatten()

                        # Iterate through 2 times the amount of slices present so that 2 plots may be made for each slice
                        for i in range(2*len(parsed)):

                            # When the index is even plot longwave
                            # The true index will be half of the passed index (0->0,2->1,4->2)
                            if i%2 == 0:
                                im = p[i].contourf(parsed[int(i/2)].lon,parsed[int(i/2)].lat,parsed[int(i/2)].LWrad.load().interpolate_na(dim='lon'),transform = ccrs.PlateCarree(),
                                    levels=np.arange(0,150,10),
                                          extend='max',
                                                   cmap = 'cividis')
                                p[i].coastlines()
                                p[i].set_title('Longwave',fontsize=20)#\n'#+titles[int(i/2)],fontsize=20)

                            # When the index is even plot shortwave
                            # The true index is the passed half the passed index rounded down [I did it more complicated than this] (1->0, 3->1, 5->2)
                            else:
                                im = p[i].contourf(parsed[i-int(math.ceil(i/2))].lon,parsed[i-int(math.ceil(i/2))].lat,parsed[i-int(math.ceil(i/2))].SWrad_day.load().interpolate_na(dim='lat'),transform = ccrs.PlateCarree(),
                                    levels=np.arange(0,150,10),
                                          extend='max',
                                                  cmap = 'cividis')
                                p[i].coastlines()
                                p[i].set_title('Shortwave',fontsize=20)#\n'+titles[i-int(math.ceil(i/2))],fontsize=20)

                        cbar_ax = f.add_axes([1,.15,.05,.7])
                        cbar = f.colorbar(im,cax=cbar_ax)
                        cbar.set_label('Radiance (W '+'$m^{-2}$ '+'$sr^{-1}$)',fontsize=20)
                        cbar.ax.tick_params(labelsize=16)

                      
                    # Same as iterative plot but without math
                    else:
                        im = p[0].contourf(parsed[0].lon,parsed[0].lat,parsed[0].LWrad.load().interpolate_na(dim='lon'),transform = ccrs.PlateCarree(),
                            levels=np.arange(0,150,10),
                                          extend='max')
                        p[0].coastlines()
                        p[0].set_title('Longwave',fontsize=20)#\n'+titles[0],fontsize=20)
                        
                        im = p[1].contourf(parsed[0].lon,parsed[0].lat,parsed[0].SWrad_day.load().interpolate_na(dim='lon'),transform = ccrs.PlateCarree(),
                            levels=np.arange(0,150,10),
                                          extend='max')
                        p[1].coastlines()
                        p[1].set_title('Shortwave',fontsize=20)#\n'+titles[0],fontsize=20)

                        cbar_ax = f.add_axes([1,.15,.05,.7])
                        cbar = f.colorbar(im,cax=cbar_ax)
                        cbar.set_label('Radiance (W '+'$m^{-2}$ '+'$sr^{-1}$)',fontsize=20)
                        cbar.ax.tick_params(labelsize=16)

                    plt.tight_layout()
                        
                    plt.suptitle('Shortwave and Longwave Radiance Comparison',fontsize = 30,y = 1.05)
                       
                        
                else:
                    print('Unrecognized response. Please try again.')
                    
                # Prompt saving the plot and allow the opportunity to name said plot
                if inp.lower() in ('longwave','shortwave cumulative','shortwave day','total','shortwave and longwave','shortwave day and longwave'):
                    name = input('Would you like to save the plot?')
                    if name.lower() in ('y','yes'):
                        name = input('What would you like to name the figure?')
                        plt.savefig('plots/'+name+'.png',bbox_inches='tight',dpi=300,transparent=False)

                    # Show the plot that has been made
                    plt.show();
                    
                    inp = 'quit'
                    
                    del parsed

                    break
                
            break
                
        if inp.lower() == 'quit':
            break
            
            
    ########## Time ##########
    elif inp.lower() == 'temporal':
    # The functions for the temporal plot slicing are exactly the same except averages are taken at the lat and lon instead of time
    # Because of this, Comments will only be made for plots and in the same manner as the spatial
    
        ########## Compare VZA or Not ##########
        
        while True:
            inp = input('Would you like to compare viewing angles?')
 
            
            if inp.lower() in ('y','yes'):
                while True:
                    try:
                        inp = input('What viewing angles would you like to compare?')
                        if inp.lower() == 'quit':
                            break
                        
                        else:
                            angles = inp.split(',')
                            for i in range(len(angles)):
                                angles[i] = str(float(angles[i]))
                                all_data.append(xr.open_mfdataset('*'+angles[i]+'_ERB.nc',chunks = {'lat':10, 'lon':10, 'time':10}))
                                angles[i] = 'VZA: '+ angles[i]
                            angle_count = len(angles)
                           
                                
                            break
                            
                    except ValueError:
                        print('Please enter numeric values.')
                        
                ########## Latitudes ##########

                while True:

                    try:
                        inp = input('What latitudes would you like to look at? Please enter a comma separated list with pairs next to each other.\nEnter all for no slice.')

                        if inp.lower() == 'all':
                            for i in range(len(all_data)):
                                weights = np.cos(np.deg2rad(all_data[i].lat))  
                                lat_slice.append((all_data[i]*weights).mean('lat'))
                            lats.append('-81')
                            lats.append('81')
                            break

                        elif inp.lower() == 'quit':
                            break

                        else:
                            lats = inp.split(',')
                            for i in range(len(lats)):
                                lats[i] = int(lats[i])
                                if lats[i] > 81:
                                    lats[i] = 81
                                elif lats[i] < -81:
                                    lats[i] = -81

                            if len(lats)%2 == 0:
                                items = 0
                                for t in range(len(all_data)):
                                    for i in range(int(len(lats)/2)):
                                        j = 2*i
                                        items += 1
                                        weights = np.cos(np.deg2rad(all_data[t].lat.sel(lat = slice(lats[j],lats[j+1]))))
                                        lat_slice.append((all_data[t].sel(lat = slice(lats[j],lats[j+1]))*weights).mean('lat'))
                                break

                            else:
                                print('Pairs are required. Please try again.')

                    except ValueError:
                        print('Unrecognized response. Please try again.')

                del all_data

                if inp.lower() == 'quit':
                    break

                ########## Longitudes ##########

                while True:

                    try:
                        inp = input('What longitudes would you like to look at? Please enter a comma separated list with pairs next to each other.\nEnter all for no slice.')

                        if inp.lower() == 'all':
                            for i in range(len(lat_slice)):
                                lon_slice.append(lat_slice[i].mean('lon'))
                            lons.append('-180')
                            lons.append('180')
                            break

                        elif inp.lower() == 'quit':
                            break

                        else:
                            lons = inp.split(',')
                            for i in range(len(lons)):
                                lons[i] = int(lons[i])

                            if len(lons)%2 == 0:
                                items = 0
                                for lat in range(len(lat_slice)):
                                    for lon in range(int(len(lons)/2)):
                                        j = 2*lon
                                        lon_slice.append(lat_slice[lat].sel(lon = slice(lons[j],lons[j+1])).mean('lon'))
                                        items += 1
                                break

                            else:
                                print('Pairs are required. Please try again')

                    except ValueError:
                        print('Unrecognized response. Please try again.')

                del lat_slice

                if inp.lower() == 'quit':
                    break

                ########## Time ##########

                while True:
                    inp = input('What time would you like to look at? Enter format as YYYY,MM,DD-YYYY,MM,DD. Enter all for no slice.')

                    try:
                        if inp.lower() == 'quit':
                            break

                        elif inp.lower() != 'all':
                            times = inp.replace('-',',')
                            times = times.split(',')
                            for i in range(len(times)):
                                times[i] = int(times[i])

                            times1 = datetime.datetime(times[0],times[1],times[2])
                            times2 = datetime.datetime(times[3],times[4],times[5]) + datetime.timedelta(days = 1)

                            times = []
                            times.append(times1.strftime('%Y-%m-%d'))
                            times.append(times2.strftime('%Y-%m-%d'))


                            for i in range(len(lon_slice)):
                                time_slice.append(lon_slice[i].sel(time = slice(str(times1),str(times2))))

                            del lon_slice

                            break

                        elif inp.lower() == 'all':
                            for i in range(len(lon_slice)):
                                time_slice.append(lon_slice[i])
                                times.append(np.datetime_as_string(time_slice[0].time.isel(time = 0).values, unit = 'D'))
                                times.append(np.datetime_as_string(time_slice[0].time.isel(time = -1).values, unit = 'D'))

                            del lon_slice

                            break

                        else:
                            print('Unrecognized response.')

                    except ValueError:
                        print('Unrecognized response.')


                if inp.lower() == 'quit':
                    break


                while True:

                    inp = input('Would you like to view seasons?')

                    if inp.lower() in ('y','yes'):
                        for i in range(len(parsed)):
                            time_slice[i].groupby('time.season')


                        while True:
                            inp = input('Which seasons would you like to view? Enter a comma separated list.')

                            if inp.lower() == 'quit':
                                break

                            elif (inp.split(',')[0].lower() in ('spring', 'summer', 'fall', 'winter')):
                                items = 0
                                season_choice = inp.split(',')
                                for i in range(len(season_choice)):
                                    season_choice[i] = season_choice[i].lower()

                                if all(item in season_choice for item in ('spring', 'summer', 'fall', 'winter')):
                                    for i in range(len(time_slice)):
                                        parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'DJF'))
                                        parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'MAM'))
                                        parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'JJA'))
                                        parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'SON'))
                                        items += 4
                                    seasons.append('Dec-Feb')
                                    seasons.append('Mar-May')
                                    seasons.append('Jun-Aug')
                                    seasons.append('Sep-Nov')
                                    

                                elif all(item in season_choice for item in ('spring', 'summer', 'fall')):
                                    for i in range(len(time_slice)):
                                        parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'MAM'))
                                        parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'JJA'))
                                        parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'SON'))
                                        items += 3
                                    seasons.append('Mar-May')
                                    seasons.append('Jun-Aug')
                                    seasons.append('Sep-Nov')

                                elif all(item in season_choice for item in('spring', 'summer', 'winter')):
                                    for i in range(len(time_slice)):
                                        parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'DJF'))
                                        parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'MAM'))
                                        parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'JJA'))
                                        items += 3
                                    seasons.append('Dec-Feb')
                                    seasons.append('Mar-May')
                                    seasons.append('Jun-Aug')

                                elif all(item in season_choice for item in('spring', 'fall', 'winter')):
                                    for i in range(len(time_slice)):
                                        parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'DJF'))
                                        parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'MAM'))
                                        parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'SON'))
                                        items += 3
                                    seasons.append('Dec-Feb')
                                    seasons.append('Mar-May')
                                    seasons.append('Sep-Nov')

                                elif all(item in season_choice for item in ('summer', 'fall', 'winter')):
                                    for i in range(len(time_slice)):
                                        parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'DJF'))
                                        parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'JJA'))
                                        parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'SON'))
                                        items += 3
                                    seasons.append('Dec-Feb')
                                    seasons.append('Jun-Aug')
                                    seasons.append('Sep-Nov')

                                elif all(item in season_choice for item in ('fall', 'winter')):
                                    for i in range(len(time_slice)):
                                        parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'DJF'))
                                        parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'SON'))
                                        items += 2
                                    seasons.append('Dec-Feb')
                                    seasons.append('Sep-Nov')

                                elif all(item in season_choice for item in ('summer', 'winter')):
                                    for i in range(len(time_slice)):
                                        parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'DJF'))
                                        parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'JJA'))
                                        items += 2
                                    seasons.append('Dec-Feb')
                                    seasons.append('Jun-Aug')

                                elif all(item in season_choice for item in ('summer', 'fall')):
                                    for i in range(len(time_slice)):
                                        parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'JJA'))
                                        parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'SON'))
                                        items += 2
                                    seasons.append('Jun-Aug')
                                    seasons.append('Sep-Nov')

                                elif all(item in season_choice for item in ('spring', 'winter')):
                                    for i in range(len(time_slice)):
                                        parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'DJF'))
                                        parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'MAM'))
                                        items += 2
                                    seasons.append('Dec-Feb')
                                    seasons.append('Mar-May')

                                elif all(item in season_choice for item in ('spring','fall')):
                                    for i in range(len(time_slice)):
                                        parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'MAM'))
                                        parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'SON'))
                                        items += 2
                                    seasons.append('Mar-May')
                                    seasons.append('Sep-Nov')

                                elif all(item in season_choice for item in ('spring', 'summer')):
                                    for i in range(len(time_slice)):
                                        parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'MAM'))
                                        parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'JJA'))
                                        items += 2
                                    seasons.append('Mar-May')
                                    seasons.append('Jun-Aug')

                                elif ('spring') in season_choice:
                                    for i in range(len(time_slice)):
                                        parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'MAM'))
                                    seasons.append('Mar-May')

                                elif ('summer') in season_choice:
                                    for i in range(len(time_slice)):
                                        parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'JJA'))
                                    seasons.append('Jun-Aug')

                                elif ('fall') in season_choice:
                                    for i in range(len(time_slice)):
                                        parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'SON'))
                                    seasons.append('Sep-Nov')

                                elif ('winter') in season_choice:
                                    for i in range(len(time_slice)):
                                        parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'DJF'))
                                    seasons.append('Dec-Feb')

                                del time_slice



                                break

                            break


                    elif inp.lower() in ('n','no'):
                        for i in range(len(time_slice)):
                            parsed.append(time_slice[i])

                        del time_slice


                    elif inp.lower() == 'quit':
                        break

                    else:
                        print('Unrecognized response.')

                    if inp.lower() == 'quit':
                        break

                    ########## Plot ##########  
                    
                    inp = input('Would you like to plot?')
                    
                    if inp.lower() in ('y','yes'):
                        for i in range(len(lats)):
                            lats[i] = str(lats[i])
                        for i in range(len(lons)):
                            lons[i] = str(lons[i])

                        if len(seasons)>0:
                            for i in range(int(len(lats)/2)):
                                for k in range(int(len(lons)/2)):
                                    for j in range(len(seasons)):
                                        l = i*2
                                        m = k*2
                                        #titles.append('Latitudes: '+lats[l]+' - '+lats[l+1]+', Longitudes: '+lons[m]+' - '+lons[m+1]+'\nSeason: '+seasons[j])
                                        titles.append('Latitudes: '+lats[l]+' : '+lats[l+1]+'\nSeason: '+seasons[j])

                        else:
                            for i in range(int(len(lats)/2)):
                                for k in range(int(len(lons)/2)):
                                    l = i*2
                                    m = k*2
                                    #titles.append('Latitudes: '+lats[l]+' - '+lats[l+1]+', Longitudes: '+lons[m]+' - '+lons[m+1])
                                    titles.append('Latitudes: '+lats[l]+' : '+lats[l+1])

                        
                        #scale = input('What should the max y tick be?')
                        #scale = float(scale)

                        c = 2 if len(parsed)/angle_count >=2 else int(len(parsed)/angle_count)
                        r = math.ceil(len(parsed)/angle_count/c)

                        f, p = plt.subplots(nrows=r,ncols=c,figsize=(20,10))

                        if c > 1:
                            p = p.flatten()

                            for i in range(int(len(parsed)/angle_count)):
                                for j in range(angle_count):
                                    p[i].plot(parsed[i+j*angle_count].time,parsed[i+j*angle_count].SWrad_day+parsed[i+j*angle_count].LWrad)


                                #z = np.polyfit(mdates.date2num(parsed[i].time).astype(float), parsed[i].SWrad_day+parsed[i].LWrad, 1)
                                #q = np.poly1d(z)

                                #p[i].plot(parsed[i].time, q(mdates.date2num(parsed[i].time).astype(float)),color='black',label='Trend of '+str(round(z[0],4))+'\nW '+'$m^{-2}$ '+'$sr^{-1}$')
                                #p[i].legend(prop = {'size':16},loc='lower right',bbox_to_anchor=(.3, 1.05))
                                p[i].tick_params(axis='both', which='major', labelsize=16)
                                p[i].tick_params(axis='both', which='minor', labelsize=16)
                                p[i].grid()

                                p[i].set_title(titles[i],fontsize=20)
                                p[i].set_xlabel('Date',fontsize=20)
                                #p[i].set_ylim((0,scale))
                                #p[i].set_ylabel('Radiance (W '+'$m^{-2}$ '+'$sr^{-1}$)',fontsize=14)
                                p[i].xaxis.set_major_locator(mdates.MonthLocator(bymonth=(1, 4, 7, 10)))
                                if len(parsed[i].time)/365 >= 4:
                                    f.autofmt_xdate()

                                f.text(-0.04, 0.5, 'Radiance (W '+'$m^{-2}$ '+'$sr^{-1}$)', va='center', rotation='vertical',fontsize=20)
                                
                        else:
                            for i in range(angle_count):
                                p.plot(parsed[i].time,parsed[i].SWrad_day+parsed[i].LWrad)
                                
                            p.legend(prop = {'size':14})
                            p.tick_params(axis='both', which='major', labelsize=16)
                            p.tick_params(axis='both', which='minor', labelsize=16)

                            p.set_title(titles[0],fontsize=20)
                            p.set_xlabel('Date',fontsize=20)
                            p.set_ylabel('Radiance (W '+'$m^{-2}$ '+'$sr^{-1}$)',fontsize=20)
                            p.xaxis.set_major_locator(mdates.MonthLocator(bymonth=(1, 4, 7, 10)))
                            p.grid()
                            

                        plt.tight_layout()    
                        plt.suptitle('Time Series of Radiance Measurements\nwith Different Viewing Zenith Angles',fontsize = 30,y = 1.1)
                        f.legend(angles,prop = {'size':20})


                        if inp.lower() in ('y','yes'):
                            name = input('Would you like to save the plot?')
                            if name.lower() in ('y','yes'):
                                name = input('What would you like to name the figure?')
                                plt.savefig('plots/'+name+'.png',bbox_inches='tight',dpi=300,transparent=False)

                            plt.show();

                            inp = 'quit'

                            del parsed

                            break
                    else:
                            break
                    break

                if inp.lower() == 'quit':
                    break
                    
                
            elif inp.lower() in ('n','no'):
                
                # Open all files saved using the data_import file into one large dataset
                all_data = xr.open_mfdataset('*0.0_ERB.nc',chunks = {'lat':10, 'lon':10, 'time':10})
    
                ########## Latitudes ##########

                while True:

                    try:
                        inp = input('What latitudes would you like to look at? Please enter a comma separated list with pairs next to each other.\nEnter all for no slice.')

                        if inp.lower() == 'all':
                            weights = np.cos(np.deg2rad(all_data.lat))  
                            all_data = all_data*weights
                            lat_slice.append(all_data.mean('lat'))
                            lats.append('-81')
                            lats.append('81')
                            break

                        elif inp.lower() == 'quit':
                            break

                        else:
                            lats = inp.split(',')
                            for i in range(len(lats)):
                                lats[i] = int(lats[i])
                                if lats[i] > 81:
                                    lats[i] = 81
                                elif lats[i] < -81:
                                    lats[i] = -81

                            if len(lats)%2 == 0:
                                for i in range(int(len(lats)/2)):
                                    j = 2*i
                                    weights = np.cos(np.deg2rad(all_data.lat.sel(lat = slice(lats[j],lats[j+1]))))
                                    lat_slice.append((all_data.sel(lat = slice(lats[j],lats[j+1]))*weights).mean('lat'))
                                break

                            else:
                                print('Pairs are required. Please try again.')

                    except ValueError:
                        print('Unrecognized response. Please try again.')

                del all_data

                if inp.lower() == 'quit':
                    break

                ########## Longitudes ##########

                while True:

                    try:
                        inp = input('What longitudes would you like to look at? Please enter a comma separated list with pairs next to each other.\nEnter all for no slice.')

                        if inp.lower() == 'all':
                            for i in range(len(lat_slice)):
                                lon_slice.append(lat_slice[i].mean('lon'))
                            lons.append('-180')
                            lons.append('180')
                            break

                        elif inp.lower() == 'quit':
                            break

                        else:
                            lons = inp.split(',')
                            for i in range(len(lons)):
                                lons[i] = int(lons[i])

                            if len(lons)%2 == 0:
                                for lat in range(len(lat_slice)):
                                    for lon in range(int(len(lons)/2)):
                                        j = 2*lon
                                        lon_slice.append(lat_slice[lat].sel(lon = slice(lons[j],lons[j+1])).mean('lon'))
                                break

                            else:
                                print('Pairs are required. Please try again')

                    except ValueError:
                        print('Unrecognized response. Please try again.')

                del lat_slice

                if inp.lower() == 'quit':
                    break

                ########## Time ##########

                while True:
                    inp = input('What time would you like to look at? Enter format as YYYY,MM,DD-YYYY,MM,DD. Enter all for no slice.')

                    try:
                        if inp.lower() == 'quit':
                            break

                        elif inp.lower() != 'all':
                            times = inp.replace('-',',')
                            times = times.split(',')
                            for i in range(len(times)):
                                times[i] = int(times[i])

                            times1 = datetime.datetime(times[0],times[1],times[2])
                            times2 = datetime.datetime(times[3],times[4],times[5]) + datetime.timedelta(days = 1)

                            times = []
                            times.append(times1.strftime('%Y-%m-%d'))
                            times.append(times2.strftime('%Y-%m-%d'))


                            for i in range(len(lon_slice)):
                                time_slice.append(lon_slice[i].sel(time = slice(str(times1),str(times2))))

                            del lon_slice

                            break

                        elif inp.lower() == 'all':
                            for i in range(len(lon_slice)):
                                time_slice.append(lon_slice[i])
                                times.append(np.datetime_as_string(time_slice[0].time.isel(time = 0).values, unit = 'D'))
                                times.append(np.datetime_as_string(time_slice[0].time.isel(time = -1).values, unit = 'D'))

                            del lon_slice

                            break

                        else:
                            print('Unrecognized response.')

                    except ValueError:
                        print('Unrecognized response.')


                if inp.lower() == 'quit':
                    break


                while True:

                    inp = input('Would you like to view seasons?')

                    if inp.lower() in ('y','yes'):
                        for i in range(len(parsed)):
                            time_slice[i].groupby('time.season')


                        while True:
                            inp = input('Which seasons would you like to view? Enter a comma separated list.')

                            if inp.lower() == 'quit':
                                break

                            elif (inp.split(',')[0].lower() in ('spring', 'summer', 'fall', 'winter')):
                                season_choice = inp.split(',')
                                for i in range(len(season_choice)):
                                    season_choice[i] = season_choice[i].lower()

                                if all(item in season_choice for item in ('spring', 'summer', 'fall', 'winter')):
                                    for i in range(len(time_slice)):
                                        parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'DJF'))
                                        parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'MAM'))
                                        parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'JJA'))
                                        parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'SON'))
                                    seasons.append('Dec-Feb')
                                    seasons.append('Mar-May')
                                    seasons.append('Jun-Aug')
                                    seasons.append('Sep-Nov')

                                elif all(item in season_choice for item in ('spring', 'summer', 'fall')):
                                    for i in range(len(time_slice)):
                                        parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'MAM'))
                                        parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'JJA'))
                                        parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'SON'))
                                    seasons.append('Mar-May')
                                    seasons.append('Jun-Aug')
                                    seasons.append('Sep-Nov')

                                elif all(item in season_choice for item in('spring', 'summer', 'winter')):
                                    for i in range(len(time_slice)):
                                        parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'DJF'))
                                        parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'MAM'))
                                        parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'JJA'))
                                    seasons.append('Dec-Feb')
                                    seasons.append('Mar-May')
                                    seasons.append('Jun-Aug')

                                elif all(item in season_choice for item in('spring', 'fall', 'winter')):
                                    for i in range(len(time_slice)):
                                        parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'DJF'))
                                        parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'MAM'))
                                        parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'SON'))
                                    seasons.append('Dec-Feb')
                                    seasons.append('Mar-May')
                                    seasons.append('Sep-Nov')

                                elif all(item in season_choice for item in ('summer', 'fall', 'winter')):
                                    for i in range(len(time_slice)):
                                        parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'DJF'))
                                        parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'JJA'))
                                        parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'SON'))
                                    seasons.append('Dec-Feb')
                                    seasons.append('Jun-Aug')
                                    seasons.append('Sep-Nov')

                                elif all(item in season_choice for item in ('fall', 'winter')):
                                    for i in range(len(time_slice)):
                                        parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'DJF'))
                                        parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'SON'))
                                    seasons.append('Dec-Feb')
                                    seasons.append('Sep-Nov')

                                elif all(item in season_choice for item in ('summer', 'winter')):
                                    for i in range(len(time_slice)):
                                        parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'DJF'))
                                        parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'JJA'))
                                    seasons.append('Dec-Feb')
                                    seasons.append('Jun-Aug')

                                elif all(item in season_choice for item in ('summer', 'fall')):
                                    for i in range(len(time_slice)):
                                        parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'JJA'))
                                        parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'SON'))
                                    seasons.append('Jun-Aug')
                                    seasons.append('Sep-Nov')

                                elif all(item in season_choice for item in ('spring', 'winter')):
                                    for i in range(len(time_slice)):
                                        parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'DJF'))
                                        parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'MAM'))
                                    seasons.append('Dec-Feb')
                                    seasons.append('Mar-May')

                                elif all(item in season_choice for item in ('spring','fall')):
                                    for i in range(len(time_slice)):
                                        parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'MAM'))
                                        parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'SON'))
                                    seasons.append('Mar-May')
                                    seasons.append('Sep-Nov')

                                elif all(item in season_choice for item in ('spring', 'summer')):
                                    for i in range(len(time_slice)):
                                        parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'MAM'))
                                        parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'JJA'))
                                    seasons.append('Mar-May')
                                    seasons.append('Jun-Aug')

                                elif ('spring') in season_choice:
                                    for i in range(len(time_slice)):
                                        parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'MAM'))
                                    seasons.append('Mar-May')

                                elif ('summer') in season_choice:
                                    for i in range(len(time_slice)):
                                        parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'JJA'))
                                    seasons.append('Jun-Aug')

                                elif ('fall') in season_choice:
                                    for i in range(len(time_slice)):
                                        parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'SON'))
                                    seasons.append('Sep-Nov')

                                elif ('winter') in season_choice:
                                    for i in range(len(time_slice)):
                                        parsed.append(time_slice[i].sel(time =time_slice[i]['time.season'] == 'DJF'))
                                    seasons.append('Dec-Feb')

                                del time_slice



                                break

                            break


                    elif inp.lower() in ('n','no'):
                        for i in range(len(time_slice)):
                            parsed.append(time_slice[i])

                        del time_slice


                    elif inp.lower() == 'quit':
                        break

                    else:
                        print('Unrecognized response.')

                    if inp.lower() == 'quit':
                        break

                    ########## Plot ##########    

                    while True:
                        inp = input('What would you like to plot?\n-Longwave\n-Shortwave Cumulative\n-Shortwave Day\n-Shortwave and Longwave\n-Total\n-Shortwave plus Longwave with Shortwave and Longwave distinct (Triple Plot)\n')

                        for i in range(len(lats)):
                            lats[i] = str(lats[i])
                        for i in range(len(lons)):
                            lons[i] = str(lons[i])

                        if len(seasons)>0:
                            for i in range(int(len(lats)/2)):
                                for k in range(int(len(lons)/2)):
                                    for j in range(len(seasons)):
                                        l = i*2
                                        m = k*2
                                        #titles.append('Latitudes: '+lats[l]+' - '+lats[l+1]+', Longitudes: '+lons[m]+' - '+lons[m+1]+'\nSeason: '+seasons[j])
                                        titles.append('Latitudes: '+lats[l]+' : '+lats[l+1]+'\nSeason: '+seasons[j])

                        else:
                            for i in range(int(len(lats)/2)):
                                for k in range(int(len(lons)/2)):
                                    l = i*2
                                    m = k*2
                                    #titles.append('Latitudes: '+lats[l]+' - '+lats[l+1]+', Longitudes: '+lons[m]+' - '+lons[m+1])
                                    titles.append('Latitudes: '+lats[l]+' : '+lats[l+1])

                        if inp.lower() == 'quit':
                            break

                        # This plot is the same as those that follow it except for a couple that will be specifically commented on
                        elif inp.lower() == 'longwave':

                            # This is the same method as the spatial plots
                            c = 2 if len(parsed)>2 else len(parsed)
                            r = math.ceil(len(parsed)/c)

                            f, p = plt.subplots(nrows=r,ncols=c,figsize=(20,10))

                            if len(parsed) > 1:
                                p = p.flatten()

                                # Iterate through all saved slices
                                for i in range(len(parsed)):

                                    #Plot the desired variable against the time index, add labels and a title
                                    p[i].plot(parsed[i].time,parsed[i].LWrad)
                                    p[i].set_title(titles[i],fontsize=20)
                                    p[i].set_xlabel('Date',fontsize=20)
                                    f.text(-0.04, 0.5, 'Radiance (W '+'$m^{-2}$ '+'$sr^{-1}$)', va='center', rotation='vertical',fontsize=20)
                                    #p[i].set_ylabel('Radiance (W '+'$m^{-2}$ '+'$sr^{-1}$)',fontsize=14)
                                    # Below this ensures the correct labels and provides label rotation
                                    p[i].xaxis.set_major_locator(mdates.MonthLocator(bymonth=(1, 4, 7, 10)))
                                    if len(parsed[i].time)/365 >= 4:
                                        f.autofmt_xdate()


                            # The same as iterative plotting but with one slice
                            else:
                                p.plot(parsed[0].time,parsed[0].LWrad)

                                p.set_title(titles[0],fontsize=20)
                                p.set_xlabel('Date',fontsize=20)
                                p.set_ylabel('Radiance (W '+'$m^{-2}$ '+'$sr^{-1}$)',fontsize=20)
                                p.xaxis.set_major_locator(mdates.MonthLocator(bymonth=(1, 4, 7, 10)))

                            # Pack plots close together, add figure title, set fontsize and the height of the figure title
                            plt.tight_layout()
                            plt.suptitle('Longwave Radiance',fontsize = 30,y = 1.1)


                        elif inp.lower() == 'shortwave cumulative':

                            c = 2 if len(parsed)>2 else len(parsed)
                            r = math.ceil(len(parsed)/c)

                            f, p = plt.subplots(nrows=r,ncols=c,figsize=(20,10))

                            if len(parsed) > 1:
                                p = p.flatten()

                                for i in range(len(parsed)):

                                    p[i].plot(parsed[i].time,parsed[i].SWrad)
                                    p[i].set_title(titles[i],fontsize=20)
                                    p[i].set_xlabel('Date',fontsize=20)
                                    f.text(-0.04, 0.5, 'Radiance (W '+'$m^{-2}$ '+'$sr^{-1}$)', va='center', rotation='vertical',fontsize=20)
                                    #p[i].set_ylabel('Radiance (W '+'$m^{-2}$ '+'$sr^{-1}$)',fontsize=14)
                                    p[i].xaxis.set_major_locator(mdates.MonthLocator(bymonth=(1, 4, 7, 10)))
                                    if len(parsed[i].time)/365 >= 4:
                                        f.autofmt_xdate()


                            else:
                                p.plot(parsed[0].time,parsed[0].SWrad)

                                p.set_title(titles[0],fontsize=20)
                                p.set_xlabel('Date',fontsize=20)
                                p.set_ylabel('Radiance (W '+'$m^{-2}$ '+'$sr^{-1}$)',fontsize=20)
                                p.xaxis.set_major_locator(mdates.MonthLocator(bymonth=(1, 4, 7, 10)))

                            plt.tight_layout()
                            plt.suptitle('Shortwave Radiance',fontsize = 30,y = 1.1)


                        elif inp.lower() == 'shortwave day':

                            c = 2 if len(parsed)>2 else len(parsed)
                            r = math.ceil(len(parsed)/c)

                            f, p = plt.subplots(nrows=r,ncols=c,figsize=(20,10))

                            if len(parsed) > 1:
                                p = p.flatten()

                                for i in range(len(parsed)):

                                    p[i].plot(parsed[i].time,parsed[i].SWrad_day)
                                    p[i].set_title(titles[i],fontsize=20)
                                    p[i].set_xlabel('Date',fontsize=20)
                                    f.text(-0.04, 0.5, 'Radiance (W '+'$m^{-2}$ '+'$sr^{-1}$)', va='center', rotation='vertical',fontsize=20)
                                    #p[i].set_ylabel('Radiance (W '+'$m^{-2}$ '+'$sr^{-1}$)',fontsize=14)
                                    p[i].xaxis.set_major_locator(mdates.MonthLocator(bymonth=(1, 4, 7, 10)))
                                    if len(parsed[i].time)/365 >= 4:
                                        f.autofmt_xdate()


                            else:
                                p.plot(parsed[0].time,parsed[0].SWrad_day)

                                p.set_title(titles[0],fontsize=20)
                                p.set_xlabel('Date',fontsize=20)
                                p.set_ylabel('Radiance (W '+'$m^{-2}$ '+'$sr^{-1}$)',fontsize=20)
                                p.xaxis.set_major_locator(mdates.MonthLocator(bymonth=(1, 4, 7, 10)))

                            plt.tight_layout()
                            plt.suptitle('Shortwave Radiance During the Day',fontsize = 30,y = 1.1)


                        # This method sums two variables to plot them as a single variable
                        elif inp.lower() == 'total':

                            c = 2 if len(parsed)>2 else len(parsed)
                            r = math.ceil(len(parsed)/c)

                            f, p = plt.subplots(nrows=r,ncols=c,figsize=(20,10))

                            if len(parsed) > 1:
                                p = p.flatten()

                                for i in range(len(parsed)):

                                    # Other than this line where the variables are added, this is the same
                                    p[i].plot(parsed[i].time,parsed[i].LWrad+parsed[i].SWrad)
                                    p[i].set_title(titles[i],fontsize=20)
                                    p[i].set_xlabel('Date',fontsize=20)
                                    f.text(-0.04, 0.5, 'Radiance (W '+'$m^{-2}$ '+'$sr^{-1}$)', va='center', rotation='vertical',fontsize=20)
                                    #p[i].set_ylabel('Radiance (W '+'$m^{-2}$ '+'$sr^{-1}$)',fontsize=14)
                                    p[i].xaxis.set_major_locator(mdates.MonthLocator(bymonth=(1, 4, 7, 10)))
                                    if len(parsed[i].time)/365 >= 4:
                                        f.autofmt_xdate()


                            else:
                                p.plot(parsed[0].time,parsed[0].LWrad+parsed[i].SWrad)
                                p.set_title(titles[0],fontsize=20)
                                p.set_xlabel('Date',fontsize=20)
                                p.set_ylabel('Radiance (W '+'$m^{-2}$ '+'$sr^{-1}$)',fontsize=20)
                                p.xaxis.set_major_locator(mdates.MonthLocator(bymonth=(1, 4, 7, 10)))

                            plt.tight_layout()
                            plt.suptitle('Total Radiance',fontsize = 30,y = 1.1)


                        # This method plots two variables on the same line 
                        elif inp.lower() == 'shortwave and longwave':

                            c = 2 if len(parsed)>2 else len(parsed)
                            r = math.ceil(len(parsed)/c)

                            f, p = plt.subplots(nrows=r,ncols=c,figsize=(20,10))

                            if len(parsed) > 1:
                                p = p.flatten()

                                for i in range(len(parsed)):

                                    # Create labels only for the first plot so that only one figure legend is created
                                    if i == 0:
                                        p[i].plot(parsed[i].time,parsed[i].LWrad,label='Longwave')
                                        p[i].plot(parsed[i].time,parsed[i].SWrad_day,label='Shortwave')

                                    else:
                                        p[i].plot(parsed[i].time,parsed[i].LWrad)
                                        p[i].plot(parsed[i].time,parsed[i].SWrad_day)

                                    p[i].set_title(titles[i],fontsize=20)
                                    p[i].set_xlabel('Date',fontsize=20)
                                    p[i].set_ylime((0,100))
                                    #p[i].set_ylabel('Radiance (W '+'$m^{-2}$ '+'$sr^{-1}$)',fontsize=24)
                                    p[i].xaxis.set_major_locator(mdates.MonthLocator(bymonth=(1, 4, 7, 10)))
                                    f.text(-0.04, 0.5, 'Radiance (W '+'$m^{-2}$ '+'$sr^{-1}$)', va='center', rotation='vertical',fontsize=20)
                                    if len(parsed[i].time)/365 >= 4:
                                        f.autofmt_xdate()


                            else:
                                p.plot(parsed[0].time,parsed[0].LWrad,label='Longwave')
                                p.plot(parsed[0].time,parsed[0].SWrad_day,label='Shortwave')
                                p.set_title(titles[0],fontsize=20)
                                p.set_xlabel('Date',fontsize=20)
                                p.set_ylabel('Radiance (W '+'$m^{-2}$ '+'$sr^{-1}$)',fontsize=20)
                                p.xaxis.set_major_locator(mdates.MonthLocator(bymonth=(1, 4, 7, 10)))

                            plt.tight_layout()
                            plt.suptitle('Comparison of Shortwave and Longwave Radiances',fontsize = 30,y = 1.1)
                            # Create a single legend for the whole plot
                            f.legend(prop = {'size':20})


                        # This method is the same as the last two, it plots three distinct lines where one is a sum of the other two
                        # Again only one legend is created
                        elif inp.lower() in ('total with shortwave and longwave distinct','triple','triple plot'):
                            #scale = input('What should the max y tick be?')
                            #scale = float(scale)

                            c = 2 if len(parsed)>2 else len(parsed)
                            r = math.ceil(len(parsed)/c)

                            f, p = plt.subplots(nrows=r,ncols=c,figsize=(20,10))

                            if len(parsed) > 1:
                                p = p.flatten()

                                for i in range(len(parsed)):

                                    p[i].plot(parsed[i].time,parsed[i].LWrad,color='blue')
                                    p[i].plot(parsed[i].time,parsed[i].SWrad,color='green')
                                    p[i].plot(parsed[i].time,parsed[i].SWrad+parsed[i].LWrad,color='orange')

                                    z = np.polyfit(mdates.date2num(parsed[i].time).astype(float), parsed[i].SWrad+parsed[i].LWrad, 1)
                                    q = np.poly1d(z)

                                    p[i].plot(parsed[i].time, q(mdates.date2num(parsed[i].time).astype(float)),color='black',label='Trend of '+str(round(len(parsed[i].time)*z[0]*round(365/len(parsed[i].time)),4))+'\nW '+'$m^{-2}$ '+'$sr^{-1}$'+'$yr^{-1}$')
                                    
                                    z = np.polyfit(mdates.date2num(parsed[i].time).astype(float), parsed[i].SWrad, 1)
                                    q = np.poly1d(z)

                                    p[i].plot(parsed[i].time, q(mdates.date2num(parsed[i].time).astype(float)),color='black',linestyle='dashed',label='Trend of '+str(round(len(parsed[i].time)*z[0]*round(365/len(parsed[i].time)),4))+'\nW '+'$m^{-2}$ '+'$sr^{-1}$'+'$yr^{-1}$')
                                    
                                    z = np.polyfit(mdates.date2num(parsed[i].time).astype(float), parsed[i].LWrad, 1)
                                    q = np.poly1d(z)

                                    p[i].plot(parsed[i].time, q(mdates.date2num(parsed[i].time).astype(float)),color='black',linestyle='dashdot',label='Trend of '+str(round(len(parsed[i].time)*z[0]*round(365/len(parsed[i].time)),4))+'\nW '+'$m^{-2}$ '+'$sr^{-1}$'+'$yr^{-1}$')
                                    
                                    
                                    p[i].legend(prop = {'size':16},loc='lower left',bbox_to_anchor=(0, 1.05),ncol=3)
                                    p[i].tick_params(axis='both', which='major', labelsize=16)
                                    p[i].tick_params(axis='both', which='minor', labelsize=16)
                                    p[i].grid()

                                    p[i].set_title(titles[i],fontsize=20)
                                    p[i].set_xlabel('Date',fontsize=20)
                                    #if i == 0 or i == 1:
                                    #    p[i].set_ylim((0,200))
                                    #elif i == 2 or i == 3:
                                    #    p[i].set_ylim((0,150))
                                    #elif i == 4 or i == 5:
                                    #    p[i].set_ylim((0,50))
                                    #elif i == 6 or i == 7:
                                    #    p[i].set_ylim((0,100))
                                        
                                    #p[i].set_ylabel('Radiance (W '+'$m^{-2}$ '+'$sr^{-1}$)',fontsize=14)
                                    p[i].xaxis.set_major_locator(mdates.MonthLocator(bymonth=(1, 4, 7, 10)))
                                    if len(parsed[i].time)/365 >= 4:
                                        f.autofmt_xdate()

                                    f.text(-0.04, 0.5, 'Radiance (W '+'$m^{-2}$ '+'$sr^{-1}$)', va='center', rotation='vertical',fontsize=20)




                            else:
                                p.plot(parsed[0].time,parsed[0].LWrad,color='blue',label='Longwave')
                                p.plot(parsed[0].time,parsed[0].SWrad,color='green',label='Shortwave')
                                p.plot(parsed[0].time,parsed[0].SWrad+parsed[0].LWrad,color='orange',label='Total')

                                z = np.polyfit(mdates.date2num(parsed[i].time).astype(float), parsed[i].SWrad+parsed[i].LWrad, 1)
                                q = np.poly1d(z)

                                p.plot(parsed[0].time, q(mdates.date2num(parsed[0].time).astype(float)),color='black',label='Trend of '+str(len(parsed[0].time)*z[0]*round(365/len(parsed[0].time)),4))+'\nW '+'$m^{-2}$ '+'$sr^{-1}$'+'$yr^{-1}$')

                                z = np.polyfit(mdates.date2num(parsed[0].time).astype(float), parsed[0].SWrad, 1)
                                q = np.poly1d(z)

                                p.plot(parsed[0].time, q(mdates.date2num(parsed[0].time).astype(float)),color='black',linestyle='dashed',label='Trend of '+str(rlen(parsed[0].time)*z[0]*round(365/len(parsed[0].time)),4))+'\nW '+'$m^{-2}$ '+'$sr^{-1}$'+'$yr^{-1}$')
                                    
                                z = np.polyfit(mdates.date2num(parsed[0].time).astype(float), parsed[0].LWrad, 1)
                                q = np.poly1d(z)

                                p.plot(parsed[0].time, q(mdates.date2num(parsed[0].time).astype(float)),color='black',linestyle='dashdot',label='Trend of '+str(round(len(parsed[0].time)*z[0]*round(365/len(parsed[0].time)),4))+'\nW '+'$m^{-2}$ '+'$sr^{-1}$'+'$yr^{-1}$')
                                    
                                
                                p.legend(prop = {'size':14},ncol=3)
                                p.tick_params(axis='both', which='major', labelsize=16)
                                p.tick_params(axis='both', which='minor', labelsize=16)

                                p.set_title(titles[0],fontsize=20)
                                p.set_xlabel('Date',fontsize=20)
                                p.set_ylabel('Radiance (W '+'$m^{-2}$ '+'$sr^{-1}$)',fontsize=20)
                                p.xaxis.set_major_locator(mdates.MonthLocator(bymonth=(1, 4, 7, 10)))
                                p.grid()


                            plt.tight_layout()    
                            plt.suptitle('Time Series of Outgoing Radiance with Latitude',fontsize = 30,y = 1.1)
                            f.legend(['Longwave','Shortwave','Total'],prop = {'size':20},loc='lower left',bbox_to_anchor=(0,1))



                        else:
                            print('Unrecognized response. Please try again.')


                        if inp.lower() in ('triple','triple plot','total with shortwave and longwave distinct','total','shortwave and longwave','shortwave cumulative','shortwave day','longwave'):
                            name = input('Would you like to save the plot?')
                            if name.lower() in ('y','yes'):
                                name = input('What would you like to name the figure?')
                                plt.savefig('plots/'+name+'.png',bbox_inches='tight',dpi=300,transparent=False)

                            plt.show();

                            inp = 'quit'

                            del parsed

                            break
                    break

                if inp.lower() == 'quit':
                    break
            elif inp.lower() == 'quit':
                break
        else:
            print('Unrecognized response. Please try again.')
    
    elif inp.lower() == 'quit':
        break

    else:
        print('Unrecognized response. Please try again.')
